In [1]:
import os
os.environ["BRASA_DATA_PATH"] = "D:\\brasa"

import sys
sys.path.append('..')

from datetime import datetime

import pyarrow as pa
import numpy as np
import pandas as pd

import brasa

In [2]:

man = brasa.engine.CacheManager()
_meta = man._load_meta_dict_by_id("aa39c903c869fa0e64fdbfc2b70be374")
template = brasa.engine.retrieve_template("b3-trades-intraday")
meta = brasa.engine.CacheMetadata(template.id)
meta.from_dict(_meta)

fname = meta.downloaded_files[0]
fname = man.cache_path(fname)
reader = template.reader
# converters = {"trade_time": lambda x: pa.scalar(x, pa.string()),}
converters = {"trade_time": str,}
df = pd.read_csv(fname,
                 encoding=reader.encoding,
                 header=None,
                 skiprows=reader.skip,
                 sep=reader.separator,
                 converters=converters,
                 names=reader.fields.names,
                 dtype_backend="pyarrow")

df["traded_quantity"] = pd.to_numeric(df["traded_quantity"], errors="coerce")
df["traded_price"] = pd.to_numeric(df["traded_price"].str.replace(",", "."), errors="coerce")
df["trade_date"] = pd.to_datetime(df["trade_date"] + " " + df["trade_time"], format="%Y-%m-%d %H%M%S%f", errors="coerce")
df["refdate"] = pd.to_datetime(df["refdate"])
df.head()

,refdate,symbol,update_action,traded_price,traded_quantity,trade_time,trade_id,trade_session_id,trade_date,buyer_code,seller_code
0,2023-12-13,TF583R,0,10.00,10000,031822580,10,1,2023-12-13 03:18:22.580,100,100
1,2023-12-13,WSPH24,0,4701.25,1,090000010,10,1,2023-12-13 09:00:00.010,3,3
2,2023-12-13,WSPH24,0,4701.25,1,090000010,20,1,2023-12-13 09:00:00.010,120,120
3,2023-12-13,WSPH24,0,4701.25,1,090000010,30,1,2023-12-13 09:00:00.010,120,120
4,2023-12-13,WSPH24,0,4701.25,1,090000010,40,1,2023-12-13 09:00:00.010,120,15


In [3]:
df.dtypes

refdate              datetime64[ns]
symbol              string[pyarrow]
update_action        int64[pyarrow]
traded_price        double[pyarrow]
traded_quantity      int64[pyarrow]
trade_time                   object
trade_id             int64[pyarrow]
trade_session_id     int64[pyarrow]
trade_date           datetime64[ns]
buyer_code           int64[pyarrow]
seller_code          int64[pyarrow]
dtype: object